In [2]:
import torch
import numpy as np
from scipy.spatial import distance
import os
import pandas as pd
import sys
sys.path.append('..')
import registry
import datafree

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = ["SimHei"]
%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
import matplotlib as mpl
print(mpl.get_cachedir())

/data/lijingru/.cache/matplotlib


In [1]:
!nvidia-smi

Wed Aug  2 14:02:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 33%   46C    P2    99W / 350W |  16599MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
| 36%   

In [62]:
# CIFAR10 convergence
log_files = [
    '../checkpoints/datafree-cudfkd/log-cifar10-vgg11-resnet18-cudfkd_L2_line91_dfq_agg.txt',
    '/data/lijingru/DataFree/checkpoints/datafree-probkd/log-cifar10-vgg11-resnet18-probkd_L2_line81_dfq.txt',
    '../checkpoints/datafree-cudfkd/log-cifar10-vgg11-resnet18-cudfkd_L2_line91_dfq_agg_random2.txt'
]

In [63]:
cat /data1/lijingru/PRE-DFKD/models/cifar10_vgg11_resnet18/log_cifar10_vgg11_resnet18_test3.txt

[03/19 12:08:02] cifar10-vgg11-resnet18 INFO: Successfully loaded teacher.
[03/19 12:08:02] cifar10-vgg11-resnet18 INFO: DISTILLATION STARTED
[03/19 12:08:16] cifar10-vgg11-resnet18 INFO: [Epoch 0/250] [loss_G: 2.997251] [loss_G2: -2.044263] [loss_S: 1.168929]
[03/19 12:08:16] cifar10-vgg11-resnet18 INFO: Time elapsed: 13.46 seconds
[03/19 12:12:00] cifar10-vgg11-resnet18 INFO: Test Avg. Loss: 0.003021, Accuracy: 0.201000
[03/19 12:12:13] cifar10-vgg11-resnet18 INFO: [Epoch 1/250] [loss_G: 2.226376] [loss_G2: -2.750356] [loss_S: 0.830833]
[03/19 12:12:13] cifar10-vgg11-resnet18 INFO: Time elapsed: 251.33 seconds
[03/19 12:15:58] cifar10-vgg11-resnet18 INFO: Test Avg. Loss: 0.002502, Accuracy: 0.298200
[03/19 12:16:12] cifar10-vgg11-resnet18 INFO: [Epoch 2/250] [loss_G: 1.989857] [loss_G2: -3.060511] [loss_S: 0.770391]
[03/19 12:16:12] cifar10-vgg11-resnet18 INFO: Time elapsed: 489.57 seconds
[03/19 12:19:56] cifar10-vgg11-resnet18 INFO: Test Avg. Loss: 0.002931, Accuracy: 0.303200
[03/

In [64]:
pre_dict_file = [
    '/data1/lijingru/PRE-DFKD/models/cifar10_vgg11_resnet18/log_cifar10_vgg11_resnet18_test1.txt',
    '/data1/lijingru/PRE-DFKD/models/cifar10_vgg11_resnet18/log_cifar10_vgg11_resnet18_test2.txt',
    '/data1/lijingru/PRE-DFKD/models/cifar10_vgg11_resnet18/log_cifar10_vgg11_resnet18_test3.txt',  
]

dfs = []
def get_pre_dict(file):
    f = open(file).readlines()
    res_dict = {}
    for logs in f:
        lines = logs.strip()
    #     print(lines.split(' ')[3])
        if lines.split(' ')[4] == '[Epoch':
            epoch = int(lines.split(' ')[5].split('/')[0])
        elif lines.split(' ')[4] == 'Test':
            acc = float(lines.split(' ')[-1])*100
            res_dict[epoch] = acc
    return res_dict

res_dicts_pre = []
for f in pre_dict_file:
    res_dict = get_pre_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:245], 'acc': list(res_dict.values())[:245], '方法': ['PRE-DFKD' for _ in range(len(res_dict))][:245]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs.append(dfa)
    values = list(res_dict.values())
    res_dicts_pre.append(np.array(values))
    
df_pre = pd.concat(dfs)
df_pre

,epoch,acc,方法
0,0,24.35,PRE-DFKD
1,1,44.10,PRE-DFKD
2,2,52.89,PRE-DFKD
3,3,57.64,PRE-DFKD
4,4,64.68,PRE-DFKD
...,...,...,...
240,240,86.30,PRE-DFKD
241,241,85.50,PRE-DFKD
242,242,86.51,PRE-DFKD
243,243,86.49,PRE-DFKD


In [65]:
def get_res_dict(file):
    f = open(file).readlines()
    res_dict = {}
    for logs in f:
        lines = logs.strip()
    #     print(lines.split(' ')[3])
        if lines.split(' ')[4] == '[Eval]':
            if lines.split(' ')[6].startswith('Acc'):
                epoch = int(lines.split(' ')[5].split('=')[-1])
                acc = float(lines.split(' ')[6].split('=')[-1])
                res_dict[epoch] = acc
    return res_dict


In [66]:
import numpy as np
import pandas as pd
res_dicts_currkd = []

for f in log_files:
    res_dict = get_res_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:245], 'acc': list(res_dict.values())[:245], '方法': ['CuDFKD（本章方法）' for _ in range(len(res_dict))][:245]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs.append(dfa)
    values = list(res_dict.values())
    res_dicts_currkd.append(np.array(values))
#     epochs.append()
df_currkd = pd.concat(dfs)
df_currkd.head()

,epoch,acc,方法
0,0,24.35,PRE-DFKD
1,1,44.10,PRE-DFKD
2,2,52.89,PRE-DFKD
3,3,57.64,PRE-DFKD
4,4,64.68,PRE-DFKD


In [67]:
import pandas as pd
# file_lists = [
#     '/data/lijingru/DataFree/checkpoints/datafree-dafl/log-cifar10-vgg11-resnet18-dafl_interval_2.txt', 
#     '../checkpoints/datafree-adadfkd/log-cifar10-vgg11-resnet18-infonce_s_exp3_1.txt',
#     '../checkpoints/datafree-dfq/log-cifar10-vgg11-resnet18-dfq_test_lineb.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar10-vgg11-resnet18-infonce_s_exp3_2.txt',
#     '../checkpoints/datafree-dfq/log-cifar10-vgg11-resnet18.txt',
#     '../checkpoints/datafree-cmi/log-cifar10-vgg11-resnet18-cmi.txt',
#     '../checkpoints/datafree-cmi/log-cifar10-vgg11-resnet18-cmi_2.txt',
#     '../checkpoints/datafree-dafl/log-cifar10-vgg11-resnet18-nonnoisy_dafl_3.txt'
# ]
file_lists = [
    '/data/lijingru/DataFree/checkpoints/datafree-dafl/log-cifar10-vgg11-resnet18-dafl_interval_2.txt', 
#     '../checkpoints/datafree-adadfkd/log-cifar10-vgg11-resnet18-infonce_s_exp3_1.txt',
    '../checkpoints/datafree-dfq/log-cifar10-vgg11-resnet18-dfq_test_lineb.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar10-vgg11-resnet18-infonce_s_exp3_2.txt',
    '../checkpoints/datafree-dfq/log-cifar10-vgg11-resnet18.txt',
    '../checkpoints/datafree-cmi/log-cifar10-vgg11-resnet18-cmi.txt',
    '../checkpoints/datafree-cmi/log-cifar10-vgg11-resnet18-cmi_2.txt',
    '../checkpoints/datafree-dafl/log-cifar10-vgg11-resnet18-nonnoisy_dafl_3.txt'
]
methods = ['DAFL','DFQ','DFQ', 'CMI','CMI', 'DAFL']
# methods = ['DAFL', 'AdaDFKD（本章方法）','DFQ','AdaDFKD（本章方法）','DFQ', 'CMI','CMI', 'DAFL']
# currkd_res = all_res[0]
res_dicts = dfs
for f, m in zip(file_lists, methods):
    res_dict = get_res_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:245], 'acc': list(res_dict.values())[:245], '方法': [m for _ in range(len(res_dict))][:245]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs.append(dfa)
    values = list(res_dict.values())
    print(len(values))

all_dfs = pd.concat(dfs)
all_dfs.head()

250
250
250
250
250
250


,epoch,acc,方法
0,0,24.35,PRE-DFKD
1,1,44.10,PRE-DFKD
2,2,52.89,PRE-DFKD
3,3,57.64,PRE-DFKD
4,4,64.68,PRE-DFKD


In [68]:
import seaborn as sns
sns.set(font='SimHei')
fig = sns.lineplot(data=all_dfs, x='epoch', y='acc', hue='方法')
x_ticks = np.arange(0, 251, 50)
y_ticks = np.arange(0, 101, 20)
fig.set_xlim(0, 255)
fig.set_ylim(0, 100)
fig.set_xlabel('epoch', fontsize=14)
fig.set_ylabel('Acc@1', fontsize=14)
fig.set_xticklabels(x_ticks, fontsize=12)
fig.set_yticklabels(y_ticks, fontsize=12)
plt.savefig('cifar10_cudfkd.pdf')

<ipython-input-68-501ef10eebf3>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  fig.set_xticklabels(x_ticks, fontsize=12)
<ipython-input-68-501ef10eebf3>:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  fig.set_yticklabels(y_ticks, fontsize=12)


<Figure size 432x288 with 1 Axes>

In [73]:
dfs_hun = []

res_dicts_pre = []
pre_dict_file = [
    '/data1/lijingru/PRE-DFKD/models/cifar100_resnet34_resnet18/log_cifar100_resnet34_resnet18_test1.txt',
    '/data1/lijingru/PRE-DFKD/models/cifar100_resnet34_resnet18/log_cifar100_resnet34_resnet18_test2.txt',
    '/data1/lijingru/PRE-DFKD/models/cifar100_resnet34_resnet18/log_cifar100_resnet34_resnet18_test3.txt',
]
for f in pre_dict_file:
    res_dict = get_pre_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:295], 'acc': list(res_dict.values())[:295], '方法': ['PRE-DFKD' for _ in range(len(res_dict))][:295]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs_hun.append(dfa)
    values = list(res_dict.values())
    res_dicts_pre.append(np.array(values))
    
df_pre = pd.concat(dfs_hun)
df_pre

,epoch,acc,方法
0,0,1.18,PRE-DFKD
1,1,1.98,PRE-DFKD
2,2,3.99,PRE-DFKD
3,3,7.28,PRE-DFKD
4,4,16.64,PRE-DFKD
...,...,...,...
290,290,70.05,PRE-DFKD
291,291,69.88,PRE-DFKD
292,292,69.51,PRE-DFKD
293,293,69.71,PRE-DFKD


In [74]:
log_files = [
    '../checkpoints/datafree-probkd/log-cifar100-resnet34-resnet18-probkd_L2_line8.txt',
    '../checkpoints/datafree-probkd/log-cifar100-resnet34-resnet18-probkd_L2_line16.txt',
    '../checkpoints/datafree-probkd/log-cifar100-resnet34-resnet18-probkd_L2_line17.txt',
    '../checkpoints/datafree-probkd/log-cifar100-resnet34-resnet18-probkd_L2_line20.txt'
]
methods = ['CMI', 'DAFL', 'ADI']
res_dicts = []

for f, m in zip(log_files, methods):
    res_dict = get_res_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:295], 'acc': list(res_dict.values())[:295], '方法': ['CuDFKD（本章方法）' for _ in range(len(res_dict))][:295]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs_hun.append(dfa)
    values = list(res_dict.values())
    print(len(values))

df_currkd_hundred = pd.concat(dfs_hun)
df_currkd_hundred.head()

300
300
300


,epoch,acc,方法
0,0,1.18,PRE-DFKD
1,1,1.98,PRE-DFKD
2,2,3.99,PRE-DFKD
3,3,7.28,PRE-DFKD
4,4,16.64,PRE-DFKD


In [75]:
# file_lists = [
#     '/data/lijingru/CMI/checkpoints/datafree-cmi/log-cifar100-resnet34-resnet18-cmi.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar100-resnet34-resnet18-infonce_res_exp3_retest4.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar100-resnet34-resnet18-infonce_res_exp3_retest10.txt',
#     '../checkpoints/datafree-dfq/log-cifar100-resnet34-resnet18-dfq_retest_line4.txt',
#     '/data/lijingru/CMI/checkpoints/datafree-dfq/log-cifar100-resnet34-resnet18-cifar100_dfq1.txt',
#     '../checkpoints/datafree-cmi/log-cifar100-resnet34-resnet18-cmi.txt',
#     '/data/lijingru/CMI/checkpoints/datafree-dafl/log-cifar100-resnet34-resnet18.txt',
#     '../checkpoints/datafree-dafl/log-cifar100-resnet34-resnet18-nonnoisy_dafl_5.txt'
# ]

# methods = ['CMI','AdaDFKD（本章方法）', 'AdaDFKD（本章方法）', 'DFQ', 'DFQ','CMI','DAFL', 'DAFL']
file_lists = [
    '/data/lijingru/CMI/checkpoints/datafree-dafl/log-cifar100-resnet34-resnet18.txt',
    '../checkpoints/datafree-dafl/log-cifar100-resnet34-resnet18-nonnoisy_dafl_5.txt',
    '../checkpoints/datafree-dfq/log-cifar100-resnet34-resnet18-dfq_retest_line4.txt',
    '/data/lijingru/CMI/checkpoints/datafree-dfq/log-cifar100-resnet34-resnet18-cifar100_dfq1.txt',
    '/data/lijingru/CMI/checkpoints/datafree-cmi/log-cifar100-resnet34-resnet18-cmi.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar100-resnet34-resnet18-infonce_res_exp3_retest4.txt',
#     '../checkpoints/datafree-adadfkd/log-cifar100-resnet34-resnet18-infonce_res_exp3_retest10.txt',
    
    '../checkpoints/datafree-cmi/log-cifar100-resnet34-resnet18-cmi.txt',
    
]

methods = ['DAFL', 'DAFL', 'DFQ', 'DFQ','CMI','CMI']
# currkd_res = all_res[0]
for f, m in zip(file_lists, methods):
    res_dict = get_res_dict(f)
    real_res_dict = {'epoch': list(res_dict.keys())[:295], 'acc': list(res_dict.values())[:295], '方法': [m for _ in range(len(res_dict))][:295]}
    dfa = pd.DataFrame(data=real_res_dict)
    dfs_hun.append(dfa)
    values = list(res_dict.values())
    print(len(values))

all_dfs = pd.concat(dfs_hun)
all_dfs

300
300
400
300
300
400


,epoch,acc,方法
0,0,1.18,PRE-DFKD
1,1,1.98,PRE-DFKD
2,2,3.99,PRE-DFKD
3,3,7.28,PRE-DFKD
4,4,16.64,PRE-DFKD
...,...,...,...
290,290,72.87,CMI
291,291,73.76,CMI
292,292,73.61,CMI
293,293,73.75,CMI


In [76]:
import seaborn as sns
sns.set(font='SimHei')
fig = sns.lineplot(data=all_dfs, x='epoch', y='acc', hue='方法')
x_ticks = np.arange(0, 300, 50)
y_ticks = np.arange(0, 80, 10)
fig.set_xlim(0, 295)
fig.set_ylim(0, 80)
fig.set_xlabel('epoch', fontsize=14)
fig.set_ylabel('Acc@1', fontsize=14)
fig.set_xticklabels(x_ticks, fontsize=12)
fig.set_yticklabels(y_ticks, fontsize=12)
plt.savefig('cifar100.pdf')

<ipython-input-76-85ada8d8fe7f>:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  fig.set_xticklabels(x_ticks, fontsize=12)
<ipython-input-76-85ada8d8fe7f>:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  fig.set_yticklabels(y_ticks, fontsize=12)


<Figure size 432x288 with 1 Axes>

In [53]:
distributed = False
gpu =3
# gpu ='0,1'
batch_size = 64
workers = 8
# num_classes = 100
# num_classes = 100
num_classes = 200
def prepare_model(model):
    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
        return model
    elif distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if gpu is not None:
#             torch.cuda.set_device(gpu)
            model.cuda()
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            batch_size = int(batch_size / 1)
            workers = int((workers + 1 - 1) / 1)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[int(x) for x in gpu.split(',')])
            return model
        else:
            model.cuda()
            model = torch.nn.parallel.DistributedDataParallel(model)
            return model
    elif gpu is not None:
        torch.cuda.set_device(gpu)
        model = model.cuda(gpu)
        return model
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        model = torch.nn.DataParallel(model).cuda()
        return model

In [4]:
from torchvision.datasets import CIFAR10,CIFAR100
import datafree
import registry
from torch import nn
student = registry.get_model('resnet18_imagenet', num_classes=num_classes)
teacher = registry.get_model('resnet50_imagenet', num_classes=num_classes, pretrained=True).eval()
# student = registry.get_model('wrn40_1', num_classes=num_classes)
# student= registry.get_model('wrn16_2', num_classes=num_classes)
# teacher = registry.get_model('wrn40_2', num_classes=num_classes)
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['tiny_imagenet'])
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar10'])
normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar100'])
student = prepare_model(student)

# teacher = teacher.to(gpu)
teacher.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = teacher.fc.in_features
teacher.fc = nn.Linear(num_ftrs, 200)
teacher.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
teacher.maxpool = nn.Sequential()
teacher = prepare_model(teacher)
ckpt = torch.load('../checkpoints/scratch/tiny_imagenet_resnet50_imagenet.pth', map_location='cpu')
dict_ckpt = dict()
for k, v in ckpt['state_dict'].items():
    dict_ckpt['.'.join(k.split('.')[1:])] = v
teacher.load_state_dict(dict_ckpt)

# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar100_vgg11.pth', map_location='cpu')['state_dict'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar10_wrn40_2.pth', map_location='cpu')['state_dict'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar100_wrn40_2.pth', map_location='cpu')['state_dict'])
print(ckpt['best_acc1'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar100_resnet34.pth', map_location='cpu')['state_dict'])
teacher.eval()


63.17


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, aff

In [10]:
# generator = datafree.models.generator.LargeGenerator(nz=512, ngf=64, img_size=32, nc=3)
# generator = prepare_model(generator)
# ckpt = torch.load('../checkpoints/datafree-deepinv/cifar100-resnet34-resnet18--deepinv_100a.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cmi/cifar10-resnet34-resnet18_adv_cmi.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-improved_cudfkd/tiny_imagenet-resnet34_imagenet-resnet18_imagenet--improved_cudfkd_exp2-R0.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/cifar100-resnet34-resnet18--probkd_L2_line4.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-resnet34-resnet18--cudfkd_L2_line33_e.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/tiny_imagenet-resnet34_imagenet-resnet18_imagenet--probkd_L3_line7-R0.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/cifar10-resnet34-resnet18--probkd_L2_line98.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/cifar10-resnet34-resnet18--probkd_L2_line73_3.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/cifar10-resnet34-resnet18--probkd_L2_line66.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-improved_cudfkd/cifar10-resnet34-resnet18--infonce_s_exp6.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-deepinv/cifar10-resnet34-resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch_i/cifar10_resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar10_wrn16_2.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar10_wrn40_1.pth', map_location='cpu')
ckpt = torch.load('../checkpoints/scratch/tiny_imagenet_resnet18_imagenet_R0.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar100_resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-wrn40_2-wrn40_1--cudfkd_L2_line33_d.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-wrn40_2-wrn16_2--cudfkd_L2_line34_c.pth', map_location='cpu')
print(ckpt['best_acc1'])
dict_ckpt = dict()
for k, v in ckpt['state_dict'].items():
    dict_ckpt['.'.join(k.split('.')[1:])] = v
# G_ckpt = ckpt['G']

student = prepare_model(student)
# student.avgpool = nn.AdaptiveAvgPool2d(1)
# num_ftrs = student.linear.in_features
# student.fc = nn.Linear(num_ftrs, 200)
# student.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
# student.maxpool = nn.Sequential()
# student.load_state_dict(ckpt['state_dict'])
student.load_state_dict(dict_ckpt)
student.eval()


43.07


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
import torchvision.transforms as T
from registry import NORMALIZE_DICT
from torchvision import datasets
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar10'])
val_transform = T.Compose([
    #T.Resize((224, 224), Image.BICUBIC),
    T.ToTensor(),
    T.Normalize( **NORMALIZE_DICT['tiny_imagenet'] ),
])
# val_transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize( **NORMALIZE_DICT['cifar100'] )]
# ) 

# val_dst = datasets.CIFAR10('~/cifar10/', train=False, download=True, transform=val_transform)
# val_dst = datasets.CIFAR100('~/cifar100/', train=False, download=True, transform=val_transform)
val_dst = datasets.ImageFolder(os.path.join('~/timagenet/tiny-imagenet-200', 'val_split'), transform=val_transform)

val_loader = torch.utils.data.DataLoader(val_dst, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [12]:
import tqdm
probs_s = []
probs_t = []
ys = []
for x, y in tqdm.tqdm(val_loader, desc='calculate_agg'):
    s_out = student(x.to(gpu))
    t_out = teacher(x.to(gpu))
#     print(np.argmax(prob_t, 1), y.numpy())
#     print(torch.sum(s_out.argmax(1).detach().cpu() == y) / 128)
    prob_s = torch.softmax(s_out.detach(), 1).cpu().numpy()
    probs_s.append(prob_s)
    prob_t = torch.softmax(t_out.detach(), 1).cpu().numpy()
    probs_t.append(prob_t)
    ys.append(y.numpy())

probs_s = np.concatenate(probs_s, 0)
probs_t = np.concatenate(probs_t, 0)
ys = np.concatenate(ys, 0)

calculate_agg: 100%|██████████| 157/157 [00:10<00:00, 14.40it/s]


In [13]:
if not os.path.exists('prob_loyalty_distribution/'):
    os.mkdir('prob_loyalty_distribution/')
# print(y)
print(np.sum(np.argmax(probs_s, 1) == ys) / len(val_dst))
print(np.sum(np.argmax(probs_t, 1) == ys) / len(val_dst))
print(np.sum(np.argmax(probs_t, 1) == np.argmax(probs_s, 1)) / len(val_dst))
dist = distance.jensenshannon(probs_s.T, probs_t.T)
dist[np.isnan(dist)] = 0 if np.sum(np.abs(probs_s[np.where(np.isnan(dist))] - probs_t[np.where(np.isnan(dist))])) < 1e-6 else 1
prob_loyalty = 1 - np.sqrt(dist)
print(np.nanmean(prob_loyalty))
# print(dist[1773])

0.4307
0.6317
0.4522
0.3556806


### Other visualization Results
The items for further visualization:
- Difficulty visualization at different timestage.
- Ablation study of $k$ and $N_{neg}$.
- Error bar of AdaDFKD, CuDFKD, CMI, ADI and DAFL.

In [51]:
# ckpt = torch.load('../checkpoints/datafree-', map_location='cpu')
# student.load_state_dict(ckpt['state_dict'])
# student.eval()
g = datafree.models.generator.DCGAN_Generator_CIFAR10(nz=512, ngf=64, nc=3, img_size=32, d=2, cond=False)
g = prepare_model(g)
g.load_state_dict(ckpt['G'])
g.eval()

DCGAN_Generator_CIFAR10(
  (project): Sequential(
    (0): Flatten()
    (1): Linear(in_features=512, out_features=16384, bias=True)
  )
  (main): Sequential(
    (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Sigmoid()
  )
)

In [58]:
import torch.nn.functional as F
ds = []
with torch.no_grad():
    for i in range(50):
        z = torch.randn(256, 512).to(gpu)
        x = g(z)
#         print(x.device)
        t_out, t_feat = teacher(normalizer(x), return_features=True)
        s_out, s_feat = student(normalizer(x), return_features=True)
        normalized_t, normalized_s = F.normalize(t_feat, dim=-1), F.normalize(s_feat, dim=-1)
    
        d = torch.mm(normalized_t, normalized_s.T)
        label = torch.arange(len(t_feat), dtype=torch.long, device=gpu)
        loss_infonce = F.cross_entropy(d / 0.07, label, reduction='none')
#         print(loss_infonce.shape)
        ds.append(loss_infonce.cpu())
#         n = d.size(0)
#         # Div(t, s) at feature map space.
#         p = torch.softmax(d / tau, 1)
print(torch.cat(ds))

tensor([5.6766, 5.4825, 5.6135,  ..., 5.6633, 5.6543, 5.6680])


In [65]:
import seaborn as sns
ds_250 = torch.cat(ds)
# sns.displot(torch.cat(ds), kind='kde')

In [75]:
ckpt = torch.load('../checkpoints/datafree-improved_cudfkd/cifar10-resnet34-resnet18-10.pth', map_location='cpu')
student.load_state_dict(ckpt['state_dict'])
g = datafree.models.generator.DCGAN_Generator_CIFAR10(nz=512, ngf=64, nc=3, img_size=32, d=2, cond=False)
g = prepare_model(g)
g.load_state_dict(ckpt['G'])
g.eval()

DCGAN_Generator_CIFAR10(
  (project): Sequential(
    (0): Flatten()
    (1): Linear(in_features=512, out_features=16384, bias=True)
  )
  (main): Sequential(
    (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Sigmoid()
  )
)

In [76]:
import torch.nn.functional as F
ds = []
with torch.no_grad():
    for i in range(50):
        z = torch.randn(256, 512).to(gpu)
        x = g(z)
#         print(x.device)
        t_out, t_feat = teacher(normalizer(x), return_features=True)
        s_out, s_feat = student(normalizer(x), return_features=True)
        normalized_t, normalized_s = F.normalize(t_feat, dim=-1), F.normalize(s_feat, dim=-1)
    
        d = torch.mm(normalized_t, normalized_s.T)
        label = torch.arange(len(t_feat), dtype=torch.long, device=gpu)
        loss_infonce = F.cross_entropy(d / 0.07, label, reduction='none')
#         print(loss_infonce.shape)
        ds.append(loss_infonce.cpu())
#         n = d.size(0)
#         # Div(t, s) at feature map space.
#         p = torch.softmax(d / tau, 1)
ds_10 = torch.cat(ds)
print(ds_10)

tensor([5.2373, 6.9553, 6.0906,  ..., 5.9348, 4.1549, 4.7906])


In [77]:
ds_dict = {
    'epoch@10': ds_10,
    'epoch@250': ds_250,
}
sns.displot(ds_dict, kind='kde')

<Figure size 455.25x360 with 1 Axes>

In [119]:
diff = []
now_epoch = 10
kl_metric = datafree.criterions.KLDiv(T=2., reduction='none')
with torch.no_grad():
    for i in range(20):
        z = torch.randn(1024, 512).to(gpu)
        x = g(z)
        t_out = teacher(normalizer(x))
        s_out = student(normalizer(x))
        kl_diff = kl_metric(t_out, s_out).sum(1).detach().cpu().numpy()
        diff.append(kl_diff)
    
diff = np.concatenate(diff)
print(diff)
with torch.no_grad():
    z = torch.randn(1024, 512).to(gpu)
    x = g(z)
    img = make_grid(x, nrow=16)
    t_out = teacher(normalizer(x))
    pesudo_label = t_out.argmax(1).cpu().numpy()
    print(pesudo_label)
    save_image(img, 'difficulty/epoch_{}.jpg'.format(now_epoch))

[0.72593755 1.0575736  1.4061728  ... 0.61674047 1.2157195  0.48502705]
[81 70 89 ...  1 19 25]


In [120]:
xs = []
for i in range(num_classes):
#     print(i)
    xs.append(x[pesudo_label == i].cpu())
    print(i, xs[i].shape)
    if xs[i].shape[0] > 0:
        img = make_grid(xs[i], nrow=1)
        save_image(img, 'difficulty/img_epoch_{}_{}.jpg'.format(now_epoch, i))

0 torch.Size([3, 3, 32, 32])
1 torch.Size([23, 3, 32, 32])
2 torch.Size([2, 3, 32, 32])
3 torch.Size([8, 3, 32, 32])
4 torch.Size([12, 3, 32, 32])
5 torch.Size([12, 3, 32, 32])
6 torch.Size([6, 3, 32, 32])
7 torch.Size([21, 3, 32, 32])
8 torch.Size([17, 3, 32, 32])
9 torch.Size([11, 3, 32, 32])
10 torch.Size([9, 3, 32, 32])
11 torch.Size([7, 3, 32, 32])
12 torch.Size([10, 3, 32, 32])
13 torch.Size([11, 3, 32, 32])
14 torch.Size([11, 3, 32, 32])
15 torch.Size([8, 3, 32, 32])
16 torch.Size([7, 3, 32, 32])
17 torch.Size([6, 3, 32, 32])
18 torch.Size([4, 3, 32, 32])
19 torch.Size([12, 3, 32, 32])
20 torch.Size([2, 3, 32, 32])
21 torch.Size([1, 3, 32, 32])
22 torch.Size([5, 3, 32, 32])
23 torch.Size([14, 3, 32, 32])
24 torch.Size([8, 3, 32, 32])
25 torch.Size([8, 3, 32, 32])
26 torch.Size([13, 3, 32, 32])
27 torch.Size([6, 3, 32, 32])
28 torch.Size([13, 3, 32, 32])
29 torch.Size([17, 3, 32, 32])
30 torch.Size([18, 3, 32, 32])
31 torch.Size([8, 3, 32, 32])
32 torch.Size([8, 3, 32, 32])
33 to

In [16]:
import json
import pandas as pd
with open('get_lambda_0207.json', 'r') as f:
    l_dict = json.load(f)
    
with open('get_lambda_rel_0207.json', 'r') as f:
    rel_dict = json.load(f)

# l_dict = {k:[v[0]]+v[5:10] for k,v in lambda_dict.items()}
# rel_dict = {k:v[:5] for k,v in lambda_dict.items()}
l_dict = pd.DataFrame(data=l_dict)
l_dict = l_dict.sort_values(by='hard')
l_dict

,N_neg,k,hard,s_nce,Acc1,Agree1,L_p
2,16384.0,0.8,0.05,0.1,66.60,71.60,0.5404
8,16384.0,0.8,0.05,0.1,67.25,72.71,0.5446
12,16384.0,0.8,0.05,0.1,65.73,71.04,0.5294
4,16384.0,0.8,0.10,0.1,67.02,72.07,0.5395
5,16384.0,0.8,0.10,0.1,66.77,72.15,0.5407
1,16384.0,0.8,0.15,0.1,66.47,71.68,0.5370
7,16384.0,0.8,0.15,0.1,66.64,71.94,0.5408
11,16384.0,0.8,0.15,0.1,65.51,70.93,0.5320
3,16384.0,0.8,0.20,0.1,65.97,71.49,0.5364
9,16384.0,0.8,0.20,0.1,66.65,71.75,0.5394


In [17]:
rel_dict = pd.DataFrame(data=rel_dict)
rel_dict = rel_dict.sort_values(by='s_nce')
rel_dict

,N_neg,k,hard,s_nce,Acc1,Agree1,L_p
17,16384.0,0.8,0.1,0.05,66.02,71.12,0.5313
12,16384.0,0.8,0.1,0.05,67.08,72.02,0.5399
4,16384.0,0.8,0.1,0.05,66.49,71.58,0.5360
7,16384.0,0.8,0.1,0.10,66.68,71.86,0.5398
5,16384.0,0.8,0.1,0.10,67.02,72.07,0.5395
6,16384.0,0.8,0.1,0.10,66.77,72.15,0.5407
13,16384.0,0.8,0.1,0.15,66.46,71.69,0.5419
0,16384.0,0.8,0.1,0.15,66.42,71.37,0.5372
8,16384.0,0.8,0.1,0.15,67.54,72.75,0.5446
16,16384.0,0.8,0.1,0.20,65.93,71.06,0.5347


In [18]:
import os
if not os.path.exists('lambda/'):
    os.mkdir('lambda/')
    
if not os.path.exists('lambda_rel/'):
    os.mkdir('lambda_rel/')

In [19]:
import seaborn as sns
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=l_dict, x='hard', y='Acc1', markers="-", palette=palette)
ax.set_xlabel(r'$\lambda$', fontsize=18)
ax.set_ylabel('Acc@1', fontsize=18)
plt.savefig('lambda/acc1.pdf')

<Figure size 432x288 with 1 Axes>

In [20]:
import seaborn as sns
sns.set(font='SimHei')
ax = sns.lineplot(data=l_dict, x='hard', y='Agree1', markers=True)
ax.set_xlabel(r'$\lambda$', fontsize=18)
ax.set_ylabel('Agree@1', fontsize=18)
plt.savefig('lambda/agree1.pdf')

<Figure size 432x288 with 1 Axes>

In [22]:
import seaborn as sns
sns.set(font='SimHei')
ax = sns.lineplot(data=l_dict, x='hard', y=f'L_p', markers="-")
ax.set_xlabel(r'$\lambda$', fontsize=18)
ax.set_ylabel(r'$L_p$', fontsize=18)
plt.savefig('lambda/lp.pdf')

<Figure size 432x288 with 1 Axes>

In [23]:
import seaborn as sns
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=rel_dict, x='s_nce', y='Acc1', markers="-", palette=palette)
ax.set_xlabel(r'$\lambda_{rel}$', fontsize=18)
ax.set_ylabel('Acc@1', fontsize=18)
plt.savefig('lambda_rel/acc1.pdf')

<Figure size 432x288 with 1 Axes>

In [24]:
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=rel_dict, x='s_nce', y='Agree1', markers="-", palette=palette)
ax.set_xlabel(r'$\lambda_{rel}$', fontsize=18)
ax.set_ylabel('Agree@1', fontsize=18)
plt.savefig('lambda_rel/agree1.pdf')

<Figure size 432x288 with 1 Axes>

In [25]:
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=rel_dict, x='s_nce', y='L_p', markers="-", palette=palette)
ax.set_xlabel(r'$\lambda_{rel}$', fontsize=18)
ax.set_ylabel(r'$L_p$', fontsize=18)
plt.savefig('lambda_rel/lp.pdf')

<Figure size 432x288 with 1 Axes>

In [26]:
if not os.path.exists('real_k/'):
    os.mkdir('real_k/')
    
with open('res_k_0203.json', 'r') as f:
    k_dict = json.load(f)

k_dict = pd.DataFrame(data=k_dict)
k_dict

,N_neg,k,hard,s_nce,Acc1,Agree1,L_p
0,16384.0,0.70,0.1,0.1,66.60,71.93,0.5391
1,16384.0,0.75,0.1,0.1,66.44,71.76,0.5378
2,16384.0,0.80,0.1,0.1,67.02,72.07,0.5395
3,16384.0,0.85,0.1,0.1,65.32,70.41,0.5273
4,16384.0,0.90,0.1,0.1,66.65,71.29,0.5345
5,16384.0,0.95,0.1,0.1,66.28,71.36,0.5362
6,16384.0,1.00,0.1,0.1,65.72,70.67,0.5301
7,16384.0,0.60,0.1,0.1,66.24,70.78,0.5289
8,16384.0,0.65,0.1,0.1,66.29,71.61,0.5372
9,16384.0,0.70,0.1,0.1,67.10,72.26,0.5438


In [27]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=k_dict, x='k', y='Acc1', markers="-", palette=palette)
ax.set_xlabel('k', fontsize=18)
ax.set_ylabel('Acc@1', fontsize=18)
plt.savefig('real_k/acc1.pdf')

<Figure size 432x288 with 1 Axes>

In [28]:
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=k_dict, x='k', y='Agree1', markers="-", palette=palette)
ax.set_xlabel('k', fontsize=18)
ax.set_ylabel('Agree@1', fontsize=18)
plt.savefig('real_k/agree1.pdf')

<Figure size 432x288 with 1 Axes>

In [29]:
sns.set(font='SimHei')
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=k_dict, x='k', y=f'L_p', markers="-", palette=palette)
ax.set_xlabel('k', fontsize=18)
ax.set_ylabel('Probability Loyalty', fontsize=18)
plt.savefig('real_k/lp.pdf')

<Figure size 432x288 with 1 Axes>

In [30]:
if not os.path.exists('Nneg/'):
    os.mkdir('Nneg/')
    
with open('res_neg_0203.json', 'r') as f:
    neg_dict = json.load(f)

neg_dict = pd.DataFrame(data=neg_dict)
neg_dict

,N_neg,k,hard,s_nce,Acc1,Agree1,L_p
0,12288.0,0.8,0.1,0.1,66.04,71.05,0.5350
1,8192.0,0.8,0.1,0.1,65.58,70.69,0.5285
2,24576.0,0.8,0.1,0.1,66.20,71.13,0.5374
3,32768.0,0.8,0.1,0.1,66.51,71.82,0.5415
4,16384.0,0.8,0.1,0.1,67.02,72.07,0.5395
5,65536.0,0.8,0.1,0.1,65.99,70.98,0.5327
6,16384.0,0.8,0.1,0.1,66.77,72.15,0.5407
7,16384.0,0.8,0.1,0.1,66.68,71.86,0.5398
8,12288.0,0.8,0.1,0.1,66.44,71.23,0.5309
9,8192.0,0.8,0.1,0.1,67.10,72.34,0.5439


In [31]:
sns.set(font='SimHei')
# plt.figure(figsize=(8,5))
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=neg_dict, x='N_neg', y='Acc1', markers="-", palette=palette)
ax.set_xlabel(r'$N_{neg}$', fontsize=18)
ax.set_ylabel('Acc@1', fontsize=18)
plt.savefig('Nneg/acc1.pdf')

<Figure size 432x288 with 1 Axes>

In [32]:
sns.set(font='SimHei')
# plt.figure(figsize=(8,5))
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=neg_dict, x='N_neg', y='Agree1', markers="-", palette=palette)
ax.set_xlabel(r'$N_{neg}$', fontsize=18)
ax.set_ylabel('Agree@1', fontsize=18)
plt.savefig('Nneg/agree1.pdf')

<Figure size 432x288 with 1 Axes>

In [33]:
sns.set(font='SimHei')
# plt.figure(figsize=(8,5))
palette = sns.color_palette("mako_r", 1)
# palette = sns.xkcd_palette(["yellow", "amber", "greyish", "faded green", "dusty purple", "red", "black"])
ax = sns.lineplot(data=neg_dict, x='N_neg', y='L_p', markers="-", palette=palette)
ax.set_xlabel(r'$N_{neg}$', fontsize=18)
ax.set_ylabel(r'$L_p$', fontsize=18)
plt.savefig('Nneg/lp.pdf')

<Figure size 432x288 with 1 Axes>

In [3]:
import os
import json
import seaborn as sns
if not os.path.exists('nt/'):
    os.mkdir('nt/')

with open('nt2_0301.json', 'r') as f:
    nt2_dict = json.load(f)

nt2_dict = pd.DataFrame(data=nt2_dict)
nt2_dict = nt2_dict.sort_values(by='Method', ascending=False)
nt2_dict_adadfkd = nt2_dict[nt2_dict['Method']=='AdaDFKD']
nt2_dict_other = nt2_dict[nt2_dict['Method']!='AdaDFKD']
nt2_dict = pd.concat([nt2_dict_other,nt2_dict_adadfkd ], 0)
nt2_dict
# nt2_dict_other = nt2_dict['方法'!='AdaDFKD']

,Noisy_mode,Method,Acc1,Agree1,L_p
48,clean,teacher,95.70,100.00,1.0000
47,$\epsilon=2$,teacher,87.51,100.00,1.0000
46,$\epsilon=1$,teacher,90.72,100.00,1.0000
45,$\epsilon=0.5$,teacher,92.81,100.00,1.0000
0,$\epsilon=0.5$,DAFL,89.34,92.63,0.7854
...,...,...,...,...,...
40,$\epsilon=1$,AdaDFKD,89.77,95.73,0.8718
39,$\epsilon=1$,AdaDFKD,89.84,96.09,0.8784
38,$\epsilon=0.5$,AdaDFKD,90.73,94.35,0.8591
37,$\epsilon=0.5$,AdaDFKD,90.57,94.68,0.8619


In [4]:
nt2_dict = nt2_dict.replace(to_replace='clean', value='无噪声教师')
nt2_dict = nt2_dict.replace(to_replace='teacher', value='教师模型')
nt2_dict

,Noisy_mode,Method,Acc1,Agree1,L_p
48,无噪声教师,教师模型,95.70,100.00,1.0000
47,$\epsilon=2$,教师模型,87.51,100.00,1.0000
46,$\epsilon=1$,教师模型,90.72,100.00,1.0000
45,$\epsilon=0.5$,教师模型,92.81,100.00,1.0000
0,$\epsilon=0.5$,DAFL,89.34,92.63,0.7854
...,...,...,...,...,...
40,$\epsilon=1$,AdaDFKD,89.77,95.73,0.8718
39,$\epsilon=1$,AdaDFKD,89.84,96.09,0.8784
38,$\epsilon=0.5$,AdaDFKD,90.73,94.35,0.8591
37,$\epsilon=0.5$,AdaDFKD,90.57,94.68,0.8619


In [5]:
sns.set(font='SimHei')
# plt.figure(figsize=(8, 5))
ax= sns.barplot(data=nt2_dict, x='Method', y='Acc1', hue='Noisy_mode', palette="Set2")
ax.set_ylim(40, 110)
ax.legend(loc='upper left', fontsize=12, ncol=2, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel('Acc@1', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([40, 50, 60, 70, 80, 90, 100], fontsize=14)
plt.savefig('nt/nt2_acc.pdf')

<Figure size 432x288 with 1 Axes>

In [6]:
sns.set(font='SimHei')
# plt.figure(figsize=(8, 5))
ax= sns.barplot(data=nt2_dict, x='Method', y='Agree1', hue='Noisy_mode', palette="Set3")
ax.set_ylim(40, 110)
ax.legend(loc='upper left', fontsize=12, ncol=4, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel('Agree@1', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([40, 50, 60, 70, 80, 90, 100], fontsize=14)
plt.savefig('nt/nt2_agree1.pdf')

<Figure size 432x288 with 1 Axes>

In [7]:
sns.set(font='SimHei')
# plt.figure(figsize=(8, 5))
ax= sns.barplot(data=nt2_dict, x='Method', y='L_p', hue='Noisy_mode', palette="icefire")
ax.set_ylim(0.3, 1.1)
ax.legend(loc='upper right', fontsize=12, ncol=2, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel(r'$L_p$', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], fontsize=14)
plt.savefig('nt/nt2_lp.pdf')

<Figure size 432x288 with 1 Axes>

In [8]:
if not os.path.exists('nt/'):
    os.mkdir('nt/')

with open('nt1_0301.json', 'r') as f:
    nt1_dict = json.load(f)

nt1_dict = pd.DataFrame(data=nt1_dict)
nt1_dict = nt1_dict.sort_values(by='Method', ascending=False)
nt1_dict_adadfkd = nt1_dict[nt1_dict['Method']=='AdaDFKD']
nt1_dict_other = nt1_dict[nt1_dict['Method']!='AdaDFKD']
nt1_dict = pd.concat([nt1_dict_other,nt1_dict_adadfkd ], 0)
nt1_dict

,Noisy_mode,Method,Acc1,Agree1,L_p
0,clean,teacher,95.70,100.00,1.0000
1,noisy,teacher,94.07,100.00,1.0000
12,clean,DAFL,91.45,93.20,0.7686
29,clean,DAFL,89.04,90.69,0.7336
28,clean,DAFL,89.41,91.06,0.7298
21,noisy,DAFL,81.86,83.55,0.6527
20,noisy,DAFL,82.89,85.03,0.6803
13,noisy,DAFL,81.66,83.85,0.6521
30,noisy,CuDFKD,92.85,96.98,0.8972
19,clean,CuDFKD,95.15,98.09,0.8897


In [9]:
nt1_dict = nt1_dict.replace(to_replace='clean', value='无噪声教师')
nt1_dict = nt1_dict.replace(to_replace='noisy', value='噪声教师')
nt1_dict = nt1_dict.replace(to_replace='teacher', value='教师模型')
nt1_dict

,Noisy_mode,Method,Acc1,Agree1,L_p
0,无噪声教师,教师模型,95.70,100.00,1.0000
1,噪声教师,教师模型,94.07,100.00,1.0000
12,无噪声教师,DAFL,91.45,93.20,0.7686
29,无噪声教师,DAFL,89.04,90.69,0.7336
28,无噪声教师,DAFL,89.41,91.06,0.7298
21,噪声教师,DAFL,81.86,83.55,0.6527
20,噪声教师,DAFL,82.89,85.03,0.6803
13,噪声教师,DAFL,81.66,83.85,0.6521
30,噪声教师,CuDFKD,92.85,96.98,0.8972
19,无噪声教师,CuDFKD,95.15,98.09,0.8897


In [10]:
# plt.figure(figsize=(8, 5))
sns.set(font='SimHei')
ax= sns.barplot(data=nt1_dict, x='Method', y='Acc1', hue='Noisy_mode', palette="plasma")
ax.set_ylim(80, 98)
ax.legend(loc='upper right', fontsize=12, ncol=2, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel('Acc@1', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([80.0, 82.5, 85, 87.5, 90, 92.5, 95, 97.5], fontsize=12)
plt.savefig('nt/acc.pdf')

<Figure size 432x288 with 1 Axes>

In [11]:
sns.set(font='SimHei')
ax= sns.barplot(data=nt1_dict, x='Method', y='Agree1', hue='Noisy_mode', palette="Set3")
ax.set_ylim(80, 102.5)
ax.legend(loc='upper right', fontsize=12, ncol=1, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel('Agree@1', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([80.0, 82.5, 85, 87.5, 90, 92.5, 95, 97.5, 100], fontsize=12)
plt.savefig('nt/agree.pdf')

<Figure size 432x288 with 1 Axes>

In [12]:
sns.set(font='SimHei')
ax= sns.barplot(data=nt1_dict, x='Method', y='L_p', hue='Noisy_mode', palette="icefire")
ax.set_ylim(0.6, 1.0)
ax.legend(loc='upper right', fontsize=12, ncol=1, columnspacing=0.2)
plt.xlabel('方法', fontsize=18)
plt.ylabel(r'$L_p$', fontsize=18)
plt.xticks(range(6), labels=['教师模型','DAFL', 'CuDFKD', 'CMI', 'ADI', 'AdaDFKD\n本章方法'], fontsize=12)
plt.yticks([0.6, 0.7, 0.8, 0.9, 1.0], fontsize=12)
plt.savefig('nt/lp.pdf')

<Figure size 432x288 with 1 Axes>